# Interacting with `.csv` files

## Setup

### Import packages for later use

`csv` is the built-in Python library for interacting with .csv files.

`os` and `shutil` are for interacting with the operating system and files.

`matplotlib` will be used for some graphs towards the end. `%matplotlib inline` tells Jupyter to show the plots next to the code cells.

In [ ]:
import csv
import os
import shutil

import matplotlib.pyplot as plt
%matplotlib inline

# Prepare paths to datasets
pokemon_csv_path = 'pokemon_data.csv'
mtg_csv_path = 'mtg_data.csv'

## Examples

### Reading .csv files

#### Counting rows

First let's do something simple: Count the lines in a .csv file.

At the very least, we'll have to open the file.

We're specifying `encoding` to not have issues with locale on Jupyter and Binder and all that.  
We're specifying `newline` per [the documentation](https://docs.python.org/3/library/csv.html#id3):  
> If newline='' is not specified, newlines embedded inside quoted fields will not be interpreted correctly, and on platforms that use \r\n linendings on write an extra \r will be added. It should always be safe to specify newline='', since the csv module does its own (universal) newline handling.

In [ ]:
pokemon_file = open(pokemon_csv_path, encoding='utf-8', newline='')

The built-in way to read .csv files is using the `csv.reader()` function

In [ ]:
csv_reader = csv.reader(pokemon_file)

Now we can use `csv_reader` as an iterable, and use it to count the lines in the file.

Each iteration will return one line of the .csv, which will then have an array of the comma-separated values

In [ ]:
# Comment out the line below to continue iterating through the csv
# You'll keep seeing this throughout the example, it resets the file position
pokemon_file.seek(0)

# next(iterable) will return the next iteration of the iterable, or raising StopIteration
# Internally, it calls the __next__() method
first_line = next(csv_reader)
print(first_line)

In [ ]:
pokemon_file.seek(0)
line_count = 0
for line in csv_reader:
    line_count += 1
    
line_count

We can also use a trick with `sum()` to count up the number of lines

In [ ]:
pokemon_file.seek(0)
line_count_with_sum = sum(1 for i in csv_reader)
line_count_with_sum

Finally, we should close the file handler to release the system resources.  
(Note: You will have to rerun the code block with `open(pokemon_csv_path, ...)` to use the above code examples again.

In [ ]:
pokemon_file.close()

To generalize this and be able to count lines for any .csv file, we can throw this logic into a function.

We also don't want to forget using `.close()`, so we could use a context manager to open the file instead.

In [ ]:
def count_csv_rows(path_to_csv):
    with open(path_to_csv, encoding='utf8', newline='') as f:
        r = csv.reader(f)
        return sum(1 for i in r)

In [ ]:
print(f'Number of rows: {count_csv_rows(pokemon_csv_path)}')

#### Finding data in a .csv

Say we want to find all the Pokemon that weigh more than 900kg.

We really want:
* A list
* of Pokemon names (a single Pokemon is 1 row, name is a single column in that row)
* that have >900 weight (weight is just another column in the row)

In [ ]:
with open(pokemon_csv_path, encoding='utf-8', newline='') as f:
    r = csv.reader(f)
    
    # Find the index of the column's we're interested in
    header_line = next(r)
    name_index = header_line.index('name')
    weight_index = header_line.index('weight')
    
    names = []
    for row in r:
        # Need to convert value to float to compare to 900
        if float(row[weight_index]) > 900:
            names.append(row[name_index])
            
    # List comprehension equivalent to the above loop, but it's not very clear what's going on
    # names = [row[name_index] for row in r if float(row[weight_index]) > 900]

print(names)

However, there is a potentially better way within the `csv` library.

By using a `csv.DictReader` instead, the first row will become our field names which we can then access directly for each row. Since boilerplate code has been reduced, the list comprehension that was a bit unwieldy above will fit nicely and be very readable.

In [ ]:
def more_than_900_weight(filename):
    with open(filename, encoding='utf-8', newline='') as f:
        r = csv.DictReader(f)
        return [row['name'] for row in r if float(row['weight']) > 900]

print(more_than_900_weight(pokemon_csv_path))

Try modifying the code above to see other information with different conditions!

#### Selecting all values in a column

Sometimes we'd like to be able to view all values in a single column, as opposed to all values in a single row.

Here's an example code block that throws in a little extra! What does `predicate` do? Why would we use it?

In [ ]:
def select_column(path_to_csv, column_name):
    with open(path_to_csv, encoding='utf-8', newline='') as f:
        r = csv.DictReader(f)
        return [row[column_name] for row in r]

In [ ]:
mega_names = select_column(pokemon_csv_path, 'megas')
print(mega_names)

There's a lot of values in the above result that we don't care about if we just want the names of the mega evolutions.

We could filter that out, literally using the `filter()` function.

In [ ]:
mega_names = select_column(pokemon_csv_path, 'megas')
mega_names = list(filter(lambda s: len(s), mega_names))
print(mega_names)

While that does work, we can make it look a bit better with a list comprehension.

In [ ]:
mega_names = select_column(pokemon_csv_path, 'megas')
mega_names = [name for name in mega_names if len(name)]
print(mega_names)

The above solution processes the entire column and then cleans the data, but what if we could do that at the same time? Then we can *maaaybe* save on some computation time and memory (do some benchmarks to see if it really makes a difference for your use case and data).
 
Let's change the `select_column()` function to allow us to only include rows that match some predicate (in our case, like a filter).

In [ ]:
def select_column(path_to_csv, column_name, predicate=None):
    with open(path_to_csv, encoding='utf-8', newline='') as f:
        r = csv.DictReader(f)
        
        if predicate:
            return [row[column_name] for row in r if predicate(row)]
        else:
            return [row[column_name] for row in r]

In [ ]:
mega_names = select_column(pokemon_csv_path, 'megas', lambda row: len(row['megas']))
print(mega_names)

#### Doing something with the column data

Let's do some more work! How about getting the average of an entire column?

In [ ]:
def get_average(path_to_csv, column_name):
    # We can take advantage of the select_column() function we already made
    # The list comprehension below maps the result to a list of floats
    values = [float(num) for num in select_column(path_to_csv, column_name)]
    
    return ### FILL IN THE CODE HERE TO GET THE AVERAGE ###

Fill in the code to return the correct value for the average. Can you do it using entirely built-in functions?

In [ ]:
# Set up list of attributes to stay DRY (Don't Repeat Yourself)
attributes = ['speed', 'health', 'attack', 'defense', 'height']

# Units of measurement (value) associated with type 
units = {'height': 'm',
         'weight': 'kg'}

for attribute in attributes:
    average = get_average(pokemon_csv_path, attribute)
    
    # Get a little more fancy by adding units to weight and height
    post_string = units[attribute] if attribute in units else ''
        
    # Take note of the :.2f down here ----v   that's like plain old printf formatting!
    print(f'Average {attribute}: {average:.2f}' + post_string)

#### Find most common occurence in a column

In the below example, we can take advantage of Python's great built-in libraries with the `collections.Counter` class. It will do most of the heavy lifting when we give it some iterable.

Documentation for the `collections` library can be found [here](https://docs.python.org/3/library/collections.html).

***WARNING***  
*In some college classes you may be restricted from using some built-in libraries, so you should still know the basics of how these work behind the scenes. You could try implementing your own Counter class!*

In [ ]:
from collections import Counter

Using the `attributes` we set up earlier and the built-in `Counter`

In [ ]:
for attribute in attributes:
    counter = Counter(select_column(pokemon_csv_path, attribute))
    
    result, count = counter.most_common(1)[0]  # most_common() returns a list, so we take the first element
    
    print(f'{result} was the most common {attribute} with {count} occurances')

Included in this repo is a larger dataset on Magic The Gathering cards, let's use our counter to look at that data too.

In [ ]:
print(f'Our Magic The Gathering dataset contains information on '
      + f'{count_csv_rows(mtg_csv_path) - 1} cards')

columns_to_count = ['type', 'subtypes']

for title in columns_to_count:
    column_data = select_column(mtg_csv_path, title)
    
    result, count = Counter(column_data).most_common(1)[0]
    print(f'"{result}" is the most common "{title}" with {count} occurances')

Hmm, that last result isn't very useful to us.

Try changing the code above and rerun to get this result:

```
"Aura" was the most common "subtypes" with 1673 occurances
```

## Writing to .csv files

### Most basic example

Writing to files follows a similar convention as reading, but we have to specify `open()`'s mode parameter. The mode defaults to read mode (`'r'`) but we want to write, so we'll use `'w'`.

To see all the `open()` modes, check out the [Python3 documentation](https://docs.python.org/3/library/functions.html#open).

In [ ]:
students = ['Bob Gel Sr.', 'Bob Gel Jr.', 'Jane Doe']
students_with_id = [(name, i) for i, name in enumerate(students)]

with open('new.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(students_with_id)  # parameter is some iterable

This next code blocks opens and confirms that we wrote the .csv as expected, and then deletes it.

In [ ]:
with open('new.csv', encoding='utf-8', newline='') as f:
    r = csv.reader(f)
    for row in r:
        print(row)
        
os.remove('new.csv')

### Modifying existing values

Going back to a previous example, let's modify our Pokemon dataset to cap the weight at 800kg.

In order to not break the earlier code blocks, we're going to first copy the dataset to a new file and operate on that.

In [ ]:
capped_pokemon_csv_path = 'pokemon_data_cap800kg.csv'
shutil.copy(pokemon_csv_path, capped_pokemon_csv_path)

Take notice of the distribution in the graph below once you run the cell.

**Run it again after we update the data and see the result of our handiwork.**

In [ ]:
weights = [float(n) for n in select_column(capped_pokemon_csv_path, 'weight')]
num_bins = 5
plt.hist(weights, num_bins, range=(700, 1000))
plt.show()

Reading from and writing to the same file at once is a recipe for disaster, so in this example we will load the entire dataset into memory first, and then write only once you have stopped reading. However, this method will have a larger memory footprint and may not work for larger datasets.

Another solution would be to first write to some temporary file, and then move that file to overwrite the original.  
*PS. There is a built-in library* `tempfile` *for this too!*

In [ ]:
fields = ''
data = []
with open(capped_pokemon_csv_path, encoding='utf-8') as f:
    r = csv.DictReader(f)
    fields = r.fieldnames
    
    for row in r:
        if float(row['weight']) > 800:
            row['weight'] = 800
        data.append(row)

with open(capped_pokemon_csv_path, 'w', encoding='utf-8') as f:
    w = csv.DictWriter(f, fields)
    w.writeheader()
    
    for row in data:
        w.writerow(row)

# Profiling Python Performance

The code blocks below can be ran to show a "pager" at the bottom of the window that displays how long that block took to run.

`%%prun` is some Jupyter (really IPython) magic that profiles the code block for us. You can also do the same for a single line by starting it with `%prun`.

For benchmarking outside of Jupyter, look into the `timeit` library ([documentation](https://docs.python.org/3/library/timeit.html)).

In [ ]:
%%prun -l 0
count_csv_rows(pokemon_csv_path) # 930 rows

In [ ]:
%%prun -l 0
count_csv_rows(mtg_csv_path) # 35758 rows

In [ ]:
%%prun -l 0
counter = Counter(select_column(mtg_csv_path, 'type'))

In [ ]:
%%prun -l 0
subtypes = select_column(mtg_csv_path, 'subtypes')
subtypes = [s for s in subtypes if len(s)]
counter = Counter(subtypes)